# 爬取豆瓣《后来的我们》热门评论

In [23]:
import requests
import pandas as pd
import numpy as np
import time
import re
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings("ignore")

设置爬虫的`cookie`和`header`,这部分需要登录豆瓣的个人账户后从*Chrome*的debugger里获取。刚开始的时候不加入`cookie`很快就被豆瓣封掉了，后来加入之后解决了。

In [2]:
url = 'https://movie.douban.com/subject/26683723/comments?status=P'

cookies = {
    'Cookie':'"ll="118281"; bid=RXLLSV3Lpl8; _pk_ref.100001.4cf6=%5B%22%22%2C%22%22%2C1525677293%2C%22https%3A%2F%2Fwww.baidu.com%2Fs%3Fie%3Dutf-8%26f%3D8%26rsv_bp%3D0%26rsv_idx%3D1%26tn%3Dbaidu%26wd%3Ddouban%26rsv_pq%3Dec71a7d500054ab9%26rsv_t%3Daea9JlVdYh1gD%252BGRz5EBNm%252FpcUlfXaX4KGe1v0B%252FlIFjMUTaJc95TIgL9nA%26rqlang%3Dcn%26rsv_enter%3D1%26rsv_sug3%3D6%26rsv_sug1%3D6%26rsv_sug7%3D100%26rsv_sug2%3D0%26inputT%3D1530%26rsv_sug4%3D1530%22%5D; _pk_id.100001.4cf6=e4478045f12b58d7.1525677293.1.1525677596.1525677293.; _pk_ses.100001.4cf6=*; __yadk_uid=sG0bTgzZcP364YKhAUooIpRTIZSaDYwU; __utma=30149280.1936213219.1525677295.1525677295.1525677295.1; __utmb=30149280.0.10.1525677295; __utmc=30149280; __utmz=30149280.1525677295.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __utma=223695111.788299203.1525677295.1525677295.1525677295.1; __utmb=223695111.0.10.1525677295; __utmc=223695111; __utmz=223695111.1525677295.1.1.utmcsr=baidu|utmccn=(organic)|utmcmd=organic|utmctr=douban; _vwo_uuid_v2=DC384E6C9A270E7570AA1C544579FBD63|b1ff83255465a52a2ca8a027f6fb3249; ps=y; ue="lixiaohuipb@163.com"; dbcl2="173117659:E2lDtbaaY9I"; ck=3lvF; push_noty_num=0; push_doumail_num=0"'
}
headers = {
    'User-Agent':"Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0"
}
sess = requests.Session()

## 加载代理IP

In [3]:
df = pd.read_csv('proxies.csv')
select_columns = ['ip', 'port', 'type']
proxies = df[select_columns]
proxies['concated'] = proxies['type'].astype(str) + "://" + proxies['ip'].astype(str) +":" + proxies['port'].astype(str)
def pick_proxy():
    proxy_dict = {}
    index = np.random.choice(proxies.shape[0])
    proxy = proxies['concated'][index]
    head = proxies.loc[index]['type']
    proxy_dict[head] = proxy
    return proxy_dict

## 获取网页内容

In [4]:
def get_content(url):
    response = sess.get(url, proxies = pick_proxy(),headers = headers, cookies = cookies, verify=False)
    content = response.text
    if response.status_code == 200:
        
        soup = BeautifulSoup(content, 'html5lib')
    else:
        print(response.status_code)
        print(content)
    return soup


## 获取包含评论的DIV

In [ ]:
def find_result(soup):
    result = soup.findAll(class_='comment')
    while result == 0:
        break
    print('We"ve found {} results in this page'.format(len(result)))
    return result

## 获取评论字段并保存到csv

In [ ]:
# extract = lambda pattern, string: re.findall(pattern, string)
# comment = [extract(commentPatter,  str(res).replace("\n", '').replace(" ", '')) for res in result]
# votes = [extract(votesPattern,  str(res).replace("\n", '').replace(" ", '')) for res in result]
# ratings = [extract(ratingsPattern,  str(res).replace("\n", '').replace(" ", '')) for res in result]
# print(len(comment), len(votes), len(ratings))

In [18]:
commentPatter = re.compile(r'<divclass="comment">.*?<pclass="">(.*?)</p>')
votesPattern = re.compile(r'<spanclass="votes">(.*?)</span>')
ratingsPattern = re.compile(r'<spanclass="allstar.*?rating"title="(.*?)"></span>')
timePattern = re.compile(r'<spanclass="comment-time".*?>(.*?)</span>')
def extract_comment(result):
    commentList = []
    for res in result:
        com = str(res).replace("\n", '').replace(" ", '')
        comment = re.findall(commentPatter, com)
        votes = re.findall(votesPattern, com)
        ratings = re.findall(ratingsPattern, com)
        time = re.findall(timePattern, com)
        ratings = ratings if ratings else ["None"]
        commentList.append((comment[0], votes[0], ratings[0], time[0]))
    df = pd.DataFrame.from_records(commentList)
    write2csv(df)

## 写入到csv文件函数

In [20]:
def write2csv(df):
    df.to_csv('comment.csv',mode='a+', header=False, encoding = 'utf-8-sig')

## 主爬虫函数

In [21]:
def scrapper(url):
    soup = get_content(url)
    result = find_result(soup)
    extract_comment(result)

## 爬取过程

In [22]:
commentCount = 500
prefix = 'https://movie.douban.com/subject/26683723/comments?start={}&limit=20&sort=new_score&status=P&percent_type='
for i in np.arange(0,commentCount,20):
    time.sleep(np.random.rand()*5)
    url = prefix.format(i)
    print("I am scrapping : " + url)
    scrapper(url)

I am scrapping : https://movie.douban.com/subject/26683723/comments?start=0&limit=20&sort=new_score&status=P&percent_type=
We"ve found 20 results in this page
I am scrapping : https://movie.douban.com/subject/26683723/comments?start=20&limit=20&sort=new_score&status=P&percent_type=
We"ve found 20 results in this page
I am scrapping : https://movie.douban.com/subject/26683723/comments?start=40&limit=20&sort=new_score&status=P&percent_type=
We"ve found 20 results in this page
I am scrapping : https://movie.douban.com/subject/26683723/comments?start=60&limit=20&sort=new_score&status=P&percent_type=
We"ve found 20 results in this page
I am scrapping : https://movie.douban.com/subject/26683723/comments?start=80&limit=20&sort=new_score&status=P&percent_type=
We"ve found 20 results in this page
I am scrapping : https://movie.douban.com/subject/26683723/comments?start=100&limit=20&sort=new_score&status=P&percent_type=
We"ve found 20 results in this page
I am scrapping : https://movie.douban.co